# Edit distance

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Task-1" data-toc-modified-id="Task-1-1">Task 1</a></span></li><li><span><a href="#Task-2" data-toc-modified-id="Task-2-2">Task 2</a></span></li><li><span><a href="#Data." data-toc-modified-id="Data.-3">Data.</a></span><ul class="toc-item"><li><span><a href="#DNA" data-toc-modified-id="DNA-3.1">DNA</a></span></li><li><span><a href="#Proteins" data-toc-modified-id="Proteins-3.2">Proteins</a></span></li></ul></li></ul></div>

Given two strings of text X and Y , there are many ways to measure how much X and Y differ.
Consider the following three operations on a string:

- D: Deletion of a character.
- I: Insertion of a character.
- S: Substitution of a character

The edit distance $d ( X , Y )$ is the minimum number of operations $\{ D , I , S \}$ needed to perform on
$X$ to produce $Y .$

**Tasks:** 

1. Design an algorithm that, given strings $X$ and $Y$ , computes the edit distance between $X$ and $Y$ . The algorithm should provide also the optimal sequence of operations transforming $X$ into $Y$.

2. Modify the previous algorithm with a penalty function: operations $D$ and $I$ have unit cost $2$, whereas operation $S$ has unit cost $1$.

## Task 1 

In [1]:
# import required package numpy as np and time
import numpy as np
import time

# We define a function `edit` that takes in two string variables (e.g. text X and Y)
# as arguments and then returns the edit distance, which is the minimum number of
# operations (D; I; S) needed to perform on X to produce Y. Furthermore, the function
# also delivers the optimal sequence of operations transforming X into Y.


def edit(str1, str2):

    # the edit distance is exclusively computed on string variables
    if not isinstance(str1, str) or not isinstance(str2, str):
        return 'the input is not string'

# if the strings are identical, then the edit distance must be zero.
    if str1 == str2:
        return 0, 'no operation required to transform'

# In order to calculate the edit distance, we will be comparing a longer string variable
# with a shorter string variable. Therefore, the code chunck below guarantees that the
# Y variable will always signify the longer string variable.

    x, y = str1, str2
    len1, len2 = len(x), len(y)

    if len1 > len2:
        x, y = y, x
        len1, len2 = len2, len1

    L = np.zeros(shape=(len1 + 1, len2 + 1))

    # To begin we generate a matrix of zeros whose values will be replaced recursively as make
    # our way through the rows of the matrix. By construction, there are `len1+1` rows, and `len2+1`
    # columns, i.e. the rows describe the characters of the string 1 variable (the shorter string)
    # and the columns represent the characters of the string 2 variable (the longer string).

    # The row, column index indicates the i'th character in the substring. For example,
    # row 3 shows the string made up of the first three characters of the string 1 variable (X), and
    # column 2 signifies the string made up of the first two characters of the string 2 variable (Y).

    # The edit distance between an empty string (no word) and a string of length n will be n, as n
    # is the number of deletions required to get from the string to an empty string. Thus, we can fill
    # up the zero-th row and column. Each entry with index (i,j) in the matrix represents the edit distance
    # between the string made up of the first i characters of the string 1 variable (X) and the string
    # made up of the first j characters of string 2 variable (Y).

    L[0, :] = np.linspace(0, len2, len2 + 1)
    L[:, 0] = np.linspace(0, len1, len1 + 1)

    # This is where we perform the central portion of the code which will help us to calculate the edit
    # distance. The edit distance between a string made up of the first i characters of x and the first j
    # characters of y (referred to as dist(i,j)) can be computed from the edit distances of the different
    # paths it took to get there.

    # If the ith character and the jth character are the same, then dist(i,j) = dist(i-1,j-1).
    # Therefore, we do not have to perform an operation on either character. However, if they are not
    # the same, then we do need to implement a substitution in order to make the ith character of the
    # first string equal to the jth character of the second string. In this case, dist(i,j) = dist(i-1,j-1) + 1.
    # This is represented by the code line np.where(x[i-1]==y[j-1],0,1)). The other two cases are as follows:
    # dist(i,j) = dist(i-1,j) + 1, since the string variable consisting of the first i characters of X can be
    # made into the string consisting of the first i-1 characters of X by deleting one character. Similarily,
    # for the substring of Y: dist(i,j) = dist(i,j-1) + 1.

    # Given that we want to employ the minimum number of operations, we must choose the minimum number of
    # these cases, which is highlighted in the equation below. Specifically, the code populates the matrix
    # with the edit distances between various substrings of X and Y and therefore obtains the edit distance
    # between X and Y in the bottom right corner of the matrix.

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            L[i, j] = min(
                L[i - 1, j] + 1, L[i, j - 1] + 1,
                L[i - 1, j - 1] + np.where(x[i - 1] == y[j - 1], 0, 1))

# this stores the optimal sequence of operations transforming X into Y
    path = []

    i = len1
    j = len2

    # Firstly, we iterate over the longer string variable Y and decide whether to keep, substitute, insert or
    # delete a character so that we end up with the smaller one.

    # When i=0, there are no options to move diagonally upwards anymore since that is the top row. Therefore,
    # no further substitutions will be required considering that a substitution requires a diagonal movement
    # upwards and to the left. As we move left across the row, if the edit distance remains the same,
    # we keep the character, and if the edit distance decreases from the jth to the j-1th character,
    # we delete a character since deletion is implied by a decrease in edit distance.

    while (i, j) != (0, 0):
        if i == 0:
            if j != 0:
                if L[i, j - 1] < L[i, j]:
                    j = j - 1
                    path.append('delete ' + y[j])
                else:
                    j = j - 1
                    path.append('keep')

# When dist(i-1,j-1) results in the minimum edit distance, and  x[i] = y[j], then we keep the jth
# character of Y.
        else:
            if L[i - 1, j - 1] + np.where(
                    x[i - 1] == y[j - 1], 0,
                    1) <= L[i, j - 1] + 1 and L[i - 1, j - 1] + np.where(
                        x[i - 1] == y[j - 1], 0, 1) <= L[i - 1, j] + 1:
                i = i - 1
                j = j - 1
                if x[i] == y[j]:
                    path.append('keep ' + y[j])

# When dist(i-1,j-1) results in the minimum edit distance, and x[i] != y[j], then we substitute
# the jth character of Y to fit the ith character of x
                else:
                    sub = 'substitute ' + y[j] + ' with ' + x[i]
                    path.append(sub)

# When the dist(i,j-1) results in the minimum edit distance, that means that a deletion of the jth
# character of Y is required.
            elif L[i, j - 1] < L[i - 1, j] and L[i, j - 1] + 1 < L[
                    i - 1, j - 1] + np.where(x[i - 1] == y[j - 1], 0, 1):
                j = j - 1
                delete = 'delete ' + y[j]
                path.append(delete)

# When the dist(i-1,j) results in the minimum edit distance, an insertion of the ith character of X is
# required.
            elif L[i - 1, j] < L[i, j - 1] and L[i - 1, j] + 1 < L[
                    i - 1, j - 1] + np.where(x[i - 1] == y[j - 1], 0, 1):
                i = i - 1
                insert = 'insert ' + x[i]
                path.append(insert)


# This returns the edit distance between X and Y along with the optimal sequence of operations
# transforming X into Y .

    return L[len1, len2], path

In [2]:
edit('reid','evelyn')

(5.0,
 ['substitute n with d',
  'substitute y with i',
  'delete l',
  'keep e',
  'substitute v with r',
  'delete e'])

In [3]:
edit('evelyn','ari')

(6.0,
 ['substitute n with i',
  'substitute y with r',
  'substitute l with a',
  'delete e',
  'delete v',
  'delete e'])

## Task 2

In [4]:
def edit_penalty(str1, str2):

    # We retain most of the code used in defining our edit function above. However, we modify the  algorithm
    # with a penalty function: operations `D` and `I` now get a unit cost of 2, whereas operation `S` now
    # get a unit cost 1.

    x, y = str1, str2
    len1, len2 = len(x), len(y)

    if len1 > len2:
        x, y = y, x
        len1, len2 = len2, len1

    L = np.zeros(shape=(len1 + 1, len2 + 1))

    # Therefore, we increase the number of operations by a mutiple of 2 for dist(0,j) and dist(i,0) so that
    # the edit distance is 2*j or 2*i as opposed to just j or i (i.e. 1*j or 1*i).

    L[0, :] = np.linspace(0, 2 * len2, len2 + 1)
    L[:, 0] = np.linspace(0, 2 * len1, len1 + 1)

    # Furthermore, now for dist(i-1,j) and dist(i,j-1), instead of only adding one, we add two since the
    # operation cost is doubled.

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            L[i, j] = min(
                L[i - 1, j] + 2, L[i, j - 1] + 2,
                L[i - 1, j - 1] + np.where(x[i - 1] == y[j - 1], 0, 1))


# Nevertheless, the algorithm works in the same way given tha the penalty only changes the edit distance
# values and not the optimal sequence search.

    path = []

    i = len1
    j = len2

    while (i, j) != (0, 0):
        if i == 0:
            if j != 0:
                if L[i, j - 1] < L[i, j]:
                    j = j - 1
                    path.append('delete ' + y[j])
                else:
                    j = j - 1
                    path.append('keep ' + y[j])

        else:
            if L[i - 1, j - 1] + np.where(
                    x[i - 1] == y[j - 1], 0,
                    1) <= L[i, j - 1] + 1 and L[i - 1, j - 1] + np.where(
                        x[i - 1] == y[j - 1], 0, 1) <= L[i - 1, j] + 1:
                i = i - 1
                j = j - 1
                if x[i] == y[j]:
                    path.append('keep ' + y[j])

                else:
                    sub = 'substitute ' + y[j] + ' with ' + x[i]
                    path.append(sub)

            elif L[i, j - 1] < L[i - 1, j] and L[i, j - 1] + 1 < L[
                    i - 1, j - 1] + np.where(x[i - 1] == y[j - 1], 0, 1):
                j = j - 1
                delete = 'delete ' + y[j]
                path.append(delete)

            elif L[i - 1, j] < L[i, j - 1] and L[i - 1, j] + 1 < L[
                    i - 1, j - 1] + np.where(x[i - 1] == y[j - 1], 0, 1):
                i = i - 1
                ins = 'insert ' + x[i]
                path.append(ins)

    return L[len1, len2], path

## Data. 

We now run both of our algorithms on the following pairs of input strings and report the edit distance along with the optimal sequence of operations.

### DNA

In [5]:
DNA_X = 'ACTACTAGATTACTTACGGATCAGGTACTTTAGAGGCTTGCAACCA'
DNA_Y = 'TACTAGCTTACTTACCCATCAGGTTTTAGAGATGGCAACCA'

In [6]:
%%time
edit(DNA_X,DNA_Y)

CPU times: user 6.67 ms, sys: 152 µs, total: 6.82 ms
Wall time: 6.71 ms


(10.0,
 ['keep A',
  'keep C',
  'keep C',
  'keep A',
  'keep A',
  'keep C',
  'keep G',
  'substitute T with G',
  'keep T',
  'substitute C with A',
  'keep G',
  'delete G',
  'keep A',
  'keep G',
  'keep A',
  'keep T',
  'keep T',
  'keep T',
  'delete C',
  'delete A',
  'keep T',
  'keep G',
  'keep G',
  'keep A',
  'keep C',
  'keep T',
  'keep A',
  'substitute G with C',
  'substitute G with C',
  'keep C',
  'keep A',
  'keep T',
  'keep T',
  'keep C',
  'keep A',
  'keep T',
  'keep T',
  'substitute A with C',
  'keep G',
  'keep A',
  'keep T',
  'keep C',
  'keep A',
  'keep T',
  'delete C',
  'delete A'])

In [7]:
%%time
edit_penalty(DNA_X, DNA_Y)

CPU times: user 9.74 ms, sys: 466 µs, total: 10.2 ms
Wall time: 9.87 ms


(15.0,
 ['keep A',
  'keep C',
  'keep C',
  'keep A',
  'keep A',
  'keep C',
  'keep G',
  'substitute T with G',
  'keep T',
  'delete C',
  'substitute G with A',
  'keep G',
  'keep A',
  'keep G',
  'keep A',
  'keep T',
  'keep T',
  'keep T',
  'delete C',
  'delete A',
  'keep T',
  'keep G',
  'keep G',
  'keep A',
  'keep C',
  'keep T',
  'keep A',
  'substitute G with C',
  'substitute G with C',
  'keep C',
  'keep A',
  'keep T',
  'keep T',
  'keep C',
  'keep A',
  'keep T',
  'keep T',
  'substitute A with C',
  'keep G',
  'keep A',
  'keep T',
  'keep C',
  'keep A',
  'keep T',
  'delete C',
  'delete A'])

### Proteins

In [8]:
PRO_X = 'AASRPRSGVPAQSDSDPCQNLAATPIPSRPPSSQSCQKCRADARQGRWGP'
PRO_Y = 'SGAPGQRGEPGPQGHAGAPGPPGPPGSDG'

In [9]:
%%time
edit(PRO_X,PRO_Y)

CPU times: user 5.91 ms, sys: 93 µs, total: 6.01 ms
Wall time: 5.95 ms


(37.0,
 ['delete P',
  'keep G',
  'delete W',
  'delete R',
  'delete G',
  'delete Q',
  'delete R',
  'delete A',
  'keep D',
  'delete A',
  'delete R',
  'delete C',
  'delete K',
  'delete Q',
  'delete C',
  'keep S',
  'substitute Q with G',
  'delete S',
  'delete S',
  'keep P',
  'keep P',
  'substitute R with G',
  'substitute S with P',
  'keep P',
  'substitute I with G',
  'keep P',
  'substitute T with A',
  'substitute A with G',
  'keep A',
  'substitute L with H',
  'substitute N with G',
  'keep Q',
  'substitute C with P',
  'substitute P with G',
  'substitute D with P',
  'substitute S with E',
  'substitute D with G',
  'substitute S with R',
  'keep Q',
  'substitute A with G',
  'keep P',
  'substitute V with A',
  'keep G',
  'keep S',
  'delete R',
  'delete P',
  'delete R',
  'delete S',
  'delete A',
  'delete A'])

In [10]:
%%time
edit_penalty(PRO_X, PRO_Y)

CPU times: user 10.9 ms, sys: 599 µs, total: 11.5 ms
Wall time: 11.3 ms


(58.0,
 ['delete P',
  'delete G',
  'delete W',
  'delete R',
  'keep G',
  'delete Q',
  'delete R',
  'delete A',
  'keep D',
  'delete A',
  'delete R',
  'delete C',
  'delete K',
  'delete Q',
  'delete C',
  'delete S',
  'delete Q',
  'keep S',
  'substitute S with G',
  'keep P',
  'keep P',
  'substitute R with G',
  'substitute S with P',
  'keep P',
  'substitute I with G',
  'keep P',
  'substitute T with A',
  'substitute A with G',
  'keep A',
  'substitute L with H',
  'substitute N with G',
  'keep Q',
  'substitute C with P',
  'substitute P with G',
  'substitute D with P',
  'substitute S with E',
  'substitute D with G',
  'substitute S with R',
  'keep Q',
  'substitute A with G',
  'keep P',
  'substitute V with A',
  'keep G',
  'delete S',
  'delete R',
  'delete P',
  'delete R',
  'keep S',
  'delete A',
  'delete A'])

**END**